In [1]:
import sklearn.cluster as cluster
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from collections import Counter

In [2]:
Crimes = pd.read_csv('crime.csv')
CrimesData = Crimes.as_matrix()
Date = CrimesData[:,[3,8]]

In [3]:
Crimes

,Dc_Dist,Psa,Dispatch_Date_Time,Dispatch_Date,Dispatch_Time,Hour,Dc_Key,Location_Block,UCR_General,Text_General_Code,Police_Districts,Month,Lon,Lat
0,18,3,2009-10-02 02:24:00,2009-10-02,14:24:00,14,200918067518,S 38TH ST / MARKETUT ST,800.0,Other Assaults,NaN,2009-10,NaN,NaN
1,14,1,2009-05-10 12:55:00,2009-05-10,00:55:00,0,200914033994,8500 BLOCK MITCH,2600.0,All Other Offenses,NaN,2009-05,NaN,NaN
2,25,J,2009-08-07 03:40:00,2009-08-07,15:40:00,15,200925083199,6TH CAMBRIA,800.0,Other Assaults,NaN,2009-08,NaN,NaN
3,35,D,2009-07-19 01:09:00,2009-07-19,01:09:00,1,200935061008,5500 BLOCK N 5TH ST,1500.0,Weapon Violations,20.0,2009-07,-75.130477,40.036389
4,9,R,2009-06-25 12:14:00,2009-06-25,00:14:00,0,200909030511,1800 BLOCK WYLIE ST,2600.0,All Other Offenses,8.0,2009-06,-75.166350,39.969532
5,17,1,2015-04-25 12:50:00,2015-04-25,12:50:00,12,201517017705,800 BLOCK S BROAD ST,600.0,Thefts,13.0,2015-04,-75.166412,39.940070
6,23,K,2009-02-10 02:33:00,2009-02-10,14:33:00,14,200923006310,2200 BLOCK RIDGE AVE,800.0,Other Assaults,16.0,2009-02,-75.171149,39.979586
7,77,A,2009-04-02 06:30:00,2009-04-02,18:30:00,18,200977001770,TERMINAL A W,500.0,Burglary Non-Residential,NaN,2009-04,NaN,NaN
8,35,D,2009-03-18 01:14:00,2009-03-18,01:14:00,1,200935021299,N 5TH SOMMERVILLE,2600.0,All Other Offenses,NaN,2009-03,NaN,NaN
9,23,L,2009-06-14 08:30:00,2009-06-14,20:30:00,20,200923029129,N 2828 W MASTER STST,2600.0,All Other Offenses,NaN,2009-06,NaN,NaN


In [3]:
def count(listOfItems, listToCount):
    countDict = {el:0 for el in listOfItems}
    for i in listToCount:
        for j in listOfItems:
            if i == j:
                countDict[j]+=1
    return countDict

In [4]:
UCR_Codes = list(range(100,2700,100))
#UCR_Codes = [500]

# Years vs Crimes

In [5]:
Pattern = []
Pattern.append(re.compile("2006"))
Pattern.append(re.compile("2007"))
Pattern.append(re.compile("2008"))
Pattern.append(re.compile("2009"))
Pattern.append(re.compile("2010"))
Pattern.append(re.compile("2011"))
Pattern.append(re.compile("2012"))
Pattern.append(re.compile("2013"))
Pattern.append(re.compile("2014"))
Pattern.append(re.compile("2015"))
DateSort = [[],[],[],[],[],[],[],[],[],[]]

j = 0;
for i in Date[:,0].flat:
    for k in range(0,9):
        if Pattern[k].match(i):
            DateSort[k].append(Date[j])
    j = j+1
        


In [6]:
plotWidth = 10

#    for j in i:
#        for k in j:
#            if type(k)==str:
#               if Pattern[0].match(k):
#                    x.append(k)
#            else:
#                y.append(k)
                
                
Arr2006 = np.array(DateSort[0])
Arr2007 = np.array(DateSort[1])
Arr2008 = np.array(DateSort[2])
Arr2009 = np.array(DateSort[2])
Arr20010 = np.array(DateSort[2])
Counts2006 = count(UCR_Codes, Arr2006[:,1])
Counts2007 = count(UCR_Codes, Arr2007[:,1])
Counts2008 = count(UCR_Codes, Arr2008[:,1])
Counts2009 = count(UCR_Codes, Arr2009[:,1])
Counts20010 = count(UCR_Codes, Arr20010[:,1])
x = UCR_Codes
y = np.array(list(Counts2006.values()))
y1 = np.array(list(Counts2007.values()))
y2 = np.array(list(Counts2008.values()))
y3 = np.array(list(Counts2009.values()))
y4 = np.array(list(Counts20010.values()))
x1 = []
x2 = []
x3 = []
x4 = []
for i in x:
    x1.append(i-plotWidth)
    x2.append(i+plotWidth)
    x3.append(i-2*plotWidth)
    x4.append(i+2*plotWidth)

In [7]:
ax = plt.subplot(111)
ax.bar(x1, y,width=plotWidth,color='b',align='center')
ax.bar(x, y1,width=plotWidth,color='g',align='center')
ax.bar(x2, y2,width=plotWidth,color='r',align='center')
ax.bar(x3, y3,width=plotWidth,color='m',align='center')
ax.bar(x4, y4,width=plotWidth,color='b',align='center')

plt.show()

# Months 2006 vs crimes

In [17]:
Pattern = []
Pattern.append(re.compile("2006-01"))
Pattern.append(re.compile("2006-02"))
Pattern.append(re.compile("2006-03"))
Pattern.append(re.compile("2006-04"))
Pattern.append(re.compile("2006-05"))
Pattern.append(re.compile("2006-06"))
Pattern.append(re.compile("2006-07"))
Pattern.append(re.compile("2006-08"))
Pattern.append(re.compile("2006-09"))
Pattern.append(re.compile("2006-10"))
Pattern.append(re.compile("2006-11"))
Pattern.append(re.compile("2006-12"))
DateSort = [[],[],[],[],[],[],[],[],[],[],[],[]]

j = 0;
for i in Date[:,0].flat:
    for k in range(0,12):
        if Pattern[k].match(i):
            DateSort[k].append(Date[j])
    j = j+1


[[array(['2006-01-09', 1100.0], dtype=object),
  array(['2006-01-31', 600.0], dtype=object),
  array(['2006-01-05', 2400.0], dtype=object),
  array(['2006-01-07', 800.0], dtype=object),
  array(['2006-01-30', 1100.0], dtype=object),
  array(['2006-01-14', 1100.0], dtype=object),
  array(['2006-01-29', 600.0], dtype=object),
  array(['2006-01-01', 300.0], dtype=object),
  array(['2006-01-02', 300.0], dtype=object),
  array(['2006-01-29', 2100.0], dtype=object),
  array(['2006-01-21', 2600.0], dtype=object),
  array(['2006-01-17', 600.0], dtype=object),
  array(['2006-01-17', 800.0], dtype=object),
  array(['2006-01-03', 1400.0], dtype=object),
  array(['2006-01-24', 1400.0], dtype=object),
  array(['2006-01-08', 1800.0], dtype=object),
  array(['2006-01-09', 600.0], dtype=object),
  array(['2006-01-22', 600.0], dtype=object),
  array(['2006-01-21', 600.0], dtype=object),
  array(['2006-01-13', 400.0], dtype=object),
  array(['2006-01-06', 1100.0], dtype=object),
  array(['2006-01-25', 3

In [23]:
plotWidth = 7

#    for j in i:
#        for k in j:
#            if type(k)==str:
#               if Pattern[0].match(k):
#                    x.append(k)
#            else:
#                y.append(k)
                
                
Arr01 = np.array(DateSort[0])
Arr02 = np.array(DateSort[1])
Arr03 = np.array(DateSort[2])
Arr04 = np.array(DateSort[3])
Arr05 = np.array(DateSort[4])
Arr06 = np.array(DateSort[5])
Arr07 = np.array(DateSort[6])
Arr08 = np.array(DateSort[7])
Arr09 = np.array(DateSort[8])
Arr10 = np.array(DateSort[9])
Arr11 = np.array(DateSort[10])
Arr12 = np.array(DateSort[11])
Counts01 = count(UCR_Codes, Arr01[:,1])
Counts02 = count(UCR_Codes, Arr02[:,1])
Counts03 = count(UCR_Codes, Arr03[:,1])
Counts04 = count(UCR_Codes, Arr04[:,1])
Counts05 = count(UCR_Codes, Arr05[:,1])
Counts06 = count(UCR_Codes, Arr06[:,1])
Counts07 = count(UCR_Codes, Arr07[:,1])
Counts08 = count(UCR_Codes, Arr08[:,1])
Counts09 = count(UCR_Codes, Arr09[:,1])
Counts10 = count(UCR_Codes, Arr10[:,1])
Counts11 = count(UCR_Codes, Arr11[:,1])
Counts12 = count(UCR_Codes, Arr12[:,1])
x = UCR_Codes
y1 = np.array(list(Counts01.values()))
y2 = np.array(list(Counts02.values()))
y3 = np.array(list(Counts03.values()))
y4 = np.array(list(Counts04.values()))
y5 = np.array(list(Counts05.values()))
y6 = np.array(list(Counts06.values()))
y7 = np.array(list(Counts07.values()))
y8 = np.array(list(Counts08.values()))
y9 = np.array(list(Counts09.values()))
y10 = np.array(list(Counts10.values()))
y11 = np.array(list(Counts11.values()))
y12 = np.array(list(Counts12.values()))
x1 = []
x2 = []
x3 = []
x4 = []
x5 = []
x6 = []
x7 = []
x8 = []
x9 = []
x10 = []
x11 = []
for i in x:
    x1.append(i-6*plotWidth)
    x2.append(i-5*plotWidth)
    x3.append(i-4*plotWidth)
    x4.append(i-3*plotWidth)
    x5.append(i-2*plotWidth)
    x6.append(i-plotWidth)
    x7.append(i+plotWidth)
    x8.append(i+2*plotWidth)
    x9.append(i+3*plotWidth)
    x10.append(i+4*plotWidth)
    x11.append(i+5*plotWidth)

In [24]:
ax = plt.subplot(111)
ax.bar(x1, y1,width=plotWidth,color='b',align='center')
ax.bar(x2, y2,width=plotWidth,color='g',align='center')
ax.bar(x3, y3,width=plotWidth,color='r',align='center')
ax.bar(x4, y4,width=plotWidth,color='m',align='center')
ax.bar(x5, y5,width=plotWidth,color='b',align='center')
ax.bar(x6, y6,width=plotWidth,color='b',align='center')
ax.bar(x, y7,width=plotWidth,color='g',align='center')
ax.bar(x7, y8,width=plotWidth,color='r',align='center')
ax.bar(x8, y9,width=plotWidth,color='m',align='center')
ax.bar(x9, y10,width=plotWidth,color='b',align='center')
ax.bar(x10, y11,width=plotWidth,color='m',align='center')
ax.bar(x11, y12,width=plotWidth,color='b',align='center')

plt.show()

# Days of December 2006 vs crimes

In [6]:
Pattern = []
Pattern.append(re.compile("2006-12-15"))
Pattern.append(re.compile("2006-12-16"))
Pattern.append(re.compile("2006-12-17"))
Pattern.append(re.compile("2006-12-18"))
Pattern.append(re.compile("2006-12-19"))
Pattern.append(re.compile("2006-12-20"))
Pattern.append(re.compile("2006-12-21"))
Pattern.append(re.compile("2006-12-22"))
Pattern.append(re.compile("2006-12-23"))
Pattern.append(re.compile("2006-12-24"))
Pattern.append(re.compile("2006-12-25"))
Pattern.append(re.compile("2006-12-26"))
Pattern.append(re.compile("2006-12-27"))
Pattern.append(re.compile("2006-12-28"))
Pattern.append(re.compile("2006-12-29"))
Pattern.append(re.compile("2006-12-30"))
Pattern.append(re.compile("2006-12-31"))
Pattern.append(re.compile("2007-01-01"))
Pattern.append(re.compile("2007-01-02"))
Pattern.append(re.compile("2007-01-03"))
Pattern.append(re.compile("2007-01-04"))
Pattern.append(re.compile("2007-01-05"))
Pattern.append(re.compile("2007-01-06"))
Pattern.append(re.compile("2007-01-07"))
Pattern.append(re.compile("2007-01-08"))
Pattern.append(re.compile("2007-01-09"))
Pattern.append(re.compile("2007-01-10"))
DateSort = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

j = 0;
for i in Date[:,0].flat:
    for k in range(0,27):
        if Pattern[k].match(i):
            DateSort[k].append(Date[j])
    j = j+1

In [7]:
plotWidth = 2


Arr01 = np.array(DateSort[0])
Arr02 = np.array(DateSort[1])
Arr03 = np.array(DateSort[2])
Arr04 = np.array(DateSort[3])
Arr05 = np.array(DateSort[4])
Arr06 = np.array(DateSort[5])
Arr07 = np.array(DateSort[6])
Arr08 = np.array(DateSort[7])
Arr09 = np.array(DateSort[8])
Arr10 = np.array(DateSort[9])
Arr11 = np.array(DateSort[10])
Arr12 = np.array(DateSort[11])
Arr13 = np.array(DateSort[12])
Arr14 = np.array(DateSort[13])
Arr15 = np.array(DateSort[14])
Arr16 = np.array(DateSort[15])
Arr17 = np.array(DateSort[16])
Arr18 = np.array(DateSort[17])
Arr19 = np.array(DateSort[18])
Arr20 = np.array(DateSort[19])
Arr21 = np.array(DateSort[20])
Arr22 = np.array(DateSort[21])
Arr23 = np.array(DateSort[22])
Arr24 = np.array(DateSort[23])
Arr25 = np.array(DateSort[24])
Arr26 = np.array(DateSort[25])
Arr27 = np.array(DateSort[26])

Counts01 = count(UCR_Codes, Arr01[:,1])
Counts02 = count(UCR_Codes, Arr02[:,1])
Counts03 = count(UCR_Codes, Arr03[:,1])
Counts04 = count(UCR_Codes, Arr04[:,1])
Counts05 = count(UCR_Codes, Arr05[:,1])
Counts06 = count(UCR_Codes, Arr06[:,1])
Counts07 = count(UCR_Codes, Arr07[:,1])
Counts08 = count(UCR_Codes, Arr08[:,1])
Counts09 = count(UCR_Codes, Arr09[:,1])
Counts10 = count(UCR_Codes, Arr10[:,1])
Counts11 = count(UCR_Codes, Arr11[:,1])
Counts12 = count(UCR_Codes, Arr12[:,1])
Counts13 = count(UCR_Codes, Arr13[:,1])
Counts14 = count(UCR_Codes, Arr14[:,1])
Counts15 = count(UCR_Codes, Arr15[:,1])
Counts16 = count(UCR_Codes, Arr16[:,1])
Counts17 = count(UCR_Codes, Arr17[:,1])
Counts18 = count(UCR_Codes, Arr18[:,1])
Counts19 = count(UCR_Codes, Arr19[:,1])
Counts20 = count(UCR_Codes, Arr20[:,1])
Counts21 = count(UCR_Codes, Arr21[:,1])
Counts22 = count(UCR_Codes, Arr22[:,1])
Counts23 = count(UCR_Codes, Arr23[:,1])
Counts24 = count(UCR_Codes, Arr24[:,1])
Counts25 = count(UCR_Codes, Arr25[:,1])
Counts26 = count(UCR_Codes, Arr26[:,1])
Counts27 = count(UCR_Codes, Arr27[:,1])

y1 = np.array(list(Counts01.values()))
y2 = np.array(list(Counts02.values()))
y3 = np.array(list(Counts03.values()))
y4 = np.array(list(Counts04.values()))
y5 = np.array(list(Counts05.values()))
y6 = np.array(list(Counts06.values()))
y7 = np.array(list(Counts07.values()))
y8 = np.array(list(Counts08.values()))
y9 = np.array(list(Counts09.values()))
y10 = np.array(list(Counts10.values()))
y11 = np.array(list(Counts11.values()))
y12 = np.array(list(Counts12.values()))
y13 = np.array(list(Counts13.values()))
y14 = np.array(list(Counts14.values()))
y15 = np.array(list(Counts15.values()))
y16 = np.array(list(Counts16.values()))
y17 = np.array(list(Counts17.values()))
y18 = np.array(list(Counts18.values()))
y19 = np.array(list(Counts19.values()))
y20 = np.array(list(Counts20.values()))
y21 = np.array(list(Counts21.values()))
y22 = np.array(list(Counts22.values()))
y23 = np.array(list(Counts23.values()))
y24 = np.array(list(Counts24.values()))
y25 = np.array(list(Counts25.values()))
y26 = np.array(list(Counts26.values()))
y27 = np.array(list(Counts27.values()))
        

x = UCR_Codes
    

x1 = []
x2 = []
x3 = []
x4 = []
x5 = []
x6 = []
x7 = []
x8 = []
x9 = []
x10 = []
x11 = []
x12 = []
x13 = []
x14 = []
x15 = []
x16 = []
x17 = []
x18 = []
x19 = []
x20 = []
x21 = []
x22 = []
x23 = []
x24 = []
x25 = []
x26 = []
for i in x:
    x1.append(i-13*plotWidth)
    x2.append(i-12*plotWidth)
    x3.append(i-11*plotWidth)
    x4.append(i-10*plotWidth)
    x5.append(i-9*plotWidth)
    x6.append(i-8*plotWidth)
    x7.append(i-7*plotWidth)
    x8.append(i-6*plotWidth)
    x9.append(i-5*plotWidth)
    x10.append(i-4*plotWidth)
    x11.append(i-3*plotWidth)
    x12.append(i-2*plotWidth)
    x13.append(i-plotWidth)
    x14.append(i+plotWidth)
    x15.append(i+2*plotWidth)
    x16.append(i+3*plotWidth)
    x17.append(i+4*plotWidth)
    x18.append(i+5*plotWidth)
    x19.append(i+6*plotWidth)
    x20.append(i+7*plotWidth)
    x21.append(i+8*plotWidth)
    x22.append(i+9*plotWidth)
    x23.append(i+10*plotWidth)
    x24.append(i+11*plotWidth)
    x25.append(i+12*plotWidth)
    x26.append(i+13*plotWidth)

In [ ]:
print(len(x2))
print(len(y2))

1
1


In [8]:
ax = plt.subplot(111)
ax.bar(x1, y1,width=plotWidth,color='b',align='center')
ax.bar(x2, y2,width=plotWidth,color='g',align='center')
ax.bar(x3, y3,width=plotWidth,color='r',align='center')
ax.bar(x4, y4,width=plotWidth,color='m',align='center')
ax.bar(x5, y5,width=plotWidth,color='b',align='center')
ax.bar(x6, y6,width=plotWidth,color='b',align='center')
ax.bar(x7, y7,width=plotWidth,color='g',align='center')
ax.bar(x8, y8,width=plotWidth,color='r',align='center')
ax.bar(x9, y9,width=plotWidth,color='m',align='center')
ax.bar(x10, y10,width=plotWidth,color='b',align='center')
ax.bar(x11, y11,width=plotWidth,color='m',align='center')
ax.bar(x12, y12,width=plotWidth,color='b',align='center')
ax.bar(x13, y13,width=plotWidth,color='b',align='center')
ax.bar(x, y14,width=plotWidth,color='g',align='center')
ax.bar(x14, y15,width=plotWidth,color='r',align='center')
ax.bar(x15, y16,width=plotWidth,color='m',align='center')
ax.bar(x16, y17,width=plotWidth,color='b',align='center')
ax.bar(x17, y18,width=plotWidth,color='b',align='center')
ax.bar(x18, y19,width=plotWidth,color='g',align='center')
ax.bar(x19, y20,width=plotWidth,color='r',align='center')
ax.bar(x20, y21,width=plotWidth,color='m',align='center')
ax.bar(x21, y22,width=plotWidth,color='b',align='center')
ax.bar(x22, y23,width=plotWidth,color='m',align='center')
ax.bar(x23, y24,width=plotWidth,color='b',align='center')
ax.bar(x24, y25,width=plotWidth,color='b',align='center')
ax.bar(x25, y26,width=plotWidth,color='m',align='center')
ax.bar(x26, y27,width=plotWidth,color='b',align='center')

plt.show()